Author: Shailendra Chalasani
Intention: Supervised Learning Likes to dislikes ratio - Decision Trees

In [34]:
import gzip
import json
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import pandas as pd
from collections import Counter, defaultdict
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [35]:
df_video_dislikes = pd.read_csv(f"../../data/cleaned_datasets/df_video_dislikes.csv")
print(len(df_video_dislikes))
df_video_dislikes.dropna(inplace=True)
print(len(df_video_dislikes))

37422
37264


In [36]:
df_video_dislikes.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'video_id', 'title', 'published_at',
       'view_count', 'likes_count', 'dislikes_count', 'comment_count', 'tags',
       'description', 'comments', 'topic1', 'topic2', 'topic3'],
      dtype='object')

In [37]:
# df_video_dislikes = df_video_dislikes[['title','view_count','likes_count','dislikes_count','comment_count','tags','description','comments']]
df_video_dislikes = df_video_dislikes[['view_count','likes_count','dislikes_count','comment_count','topic1','topic2','topic3']]
# df_video_dislikes['likes_dislikes_ratio'] = df_video_dislikes['likes_count']/df_video_dislikes['dislikes_count']
df_video_dislikes

,view_count,likes_count,dislikes_count,comment_count,topic1,topic2,topic3
0,1048888,19515,226,1319,2,9,3
1,15352638,359277,7479,18729,5,7,1
2,925281,11212,401,831,1,2,9
3,2641597,39131,441,3745,5,3,7
4,715724,32887,367,1067,5,0,1
...,...,...,...,...,...,...,...
37417,120408275,2180780,35871,81360,5,0,8
37418,2841917,20759,1049,2624,6,9,7
37419,13346678,720854,4426,90616,1,7,8
37420,252057,19198,1234,1471,9,8,0


In [38]:
from sklearn.preprocessing import StandardScaler

X = df_video_dislikes[['view_count','likes_count','comment_count','topic1','topic2','topic3']]
y =df_video_dislikes['dislikes_count']

train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=0)

scaler = StandardScaler().fit(train_X)
X_train = scaler.transform(train_X)
X_test = scaler.transform(test_X)

In [39]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeRegressor
import numpy as np

# Create the DecisionTreeRegressor object
decision_tree = DecisionTreeRegressor()

# Define the parameter grid for grid search
param_grid = {
    'max_depth': [None, 3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=decision_tree, param_grid=param_grid, scoring='r2', cv=5)

# Fit the GridSearchCV object to the data
grid_search.fit(train_X, train_y)

# Get the mean and standard deviation of the R2 scores
r2_scores = grid_search.cv_results_['mean_test_score']
r2_mean = np.mean(r2_scores)
r2_std = np.std(r2_scores)

# Get the best parameters
best_params = grid_search.best_params_

print("Mean R2 Score:", r2_mean)
print("Standard Deviation of R2 Score:", r2_std)
print("Best Parameters:", best_params)


Mean R2 Score: 0.4497216670205615
Standard Deviation of R2 Score: 0.05569635951884215
Best Parameters: {'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 2}


In the context of an R2 score of 0.71, it means that approximately 71% of the variance in the dependent variable can be explained by the independent variables included in the regression model. This indicates a relatively good fit, as a higher proportion of the variance is accounted for by the model.

In [40]:
from sklearn.inspection import permutation_importance

# Train the DecisionTreeRegressor model
dtree = DecisionTreeRegressor(min_samples_leaf=3,min_samples_split=2)
dtree.fit(train_X, train_y)

# Calculate permutation importances
result = permutation_importance(dtree, test_X, test_y, n_repeats=10)

# Get the feature importances and their indices
importances = result.importances_mean
indices = np.argsort(importances)

# Print the feature importances
for f in range(train_X.shape[1]):
    print(f"{train_X.columns[indices[f]]}: {importances[indices[f]]}")

topic2: -0.0008290474846264217
topic3: 0.0026897271395146193
topic1: 0.003856199578394659
view_count: 0.2684777358950366
likes_count: 0.31818527502591587
comment_count: 0.374615197950508


In [9]:
from sklearn.model_selection import cross_val_score

# Define the range of values for the number of min samples leaf
min_samp_leaf = [1, 3, 5,7]

# Perform sensitivity analysis by iterating over different values of neighbors
for k in min_samp_leaf:
    model = DecisionTreeRegressor(min_samples_leaf=k,min_samples_split=2)

    # Evaluate the model using cross-validation and the desired metric (e.g., mean squared error)
    scores = cross_val_score(model, train_X, train_y, cv=5, scoring='neg_mean_squared_error')

    # Calculate the mean and standard deviation of the scores
    print(scores)
    mean_score = -scores.mean()  # Since scoring='neg_mean_squared_error', we take the negative of the mean score
    std_score = scores.std()

    # Print the results
    print(f"Min samples Leaf: {k}, Mean MSE: {mean_score:.5f}, Standard Deviation: {std_score:.5f}")

[-287.82736252  -99.02661154 -655.97440589 -294.3655928  -175.03002425]
Min samples Leaf: 1, Mean MSE: 302.44480, Standard Deviation: 191.24623
[-310.21373207 -125.70050192 -581.63108479 -256.32130747 -155.57423824]
Min samples Leaf: 3, Mean MSE: 285.88817, Standard Deviation: 162.21690
[-386.80169713 -197.82820366 -605.4241006  -188.14261475 -187.29798551]
Min samples Leaf: 5, Mean MSE: 313.09892, Standard Deviation: 164.68965
[-325.84613944 -169.52517646 -654.33004733 -255.48526609 -217.81633164]
Min samples Leaf: 7, Mean MSE: 324.60059, Standard Deviation: 172.59853


In [23]:
predicted_dtree = dtree.predict(test_X)

In [25]:
from sklearn.dummy import DummyRegressor

dummy_reg_mean = DummyRegressor(strategy="mean")
dummy_reg_median = DummyRegressor(strategy="median")

dummy_reg_mean.fit(train_X,train_y)
dummy_reg_median.fit(train_X,train_y)

mean_predicted= dummy_reg_mean.predict(test_X)
median_predicted=dummy_reg_median.predict(test_X)

# dtree_r2 = r2_score(test_y, predicted_dtree)
mean_r2 = r2_score(test_y, mean_predicted)
median_r2 = r2_score(test_y, median_predicted)

print("dtree_r2=", r2_mean)
print("dummy mean_r2=", mean_r2)
print("dummy median_r2=", median_r2)

dtree_r2= 0.7060909010339231
dummy mean_r2= -9.76538597829979e-06
dummy median_r2= -0.06943460924051359


The decisionTreeRegressor does better than the dummy regressors

In [11]:
# MSE is used to evaluate the performance of estimators. A lower MSE indicates that the estimator is more accurate. MSE can also be used to compare the performance of different estimators.
mse = mean_squared_error(test_y, predicted_dtree)
mse

117.806268909707

In [12]:
#In statistics, the coefficient of determination, or R-squared, is a measure of how well a model fits a set of data. It is calculated as the ratio of the variance explained by the model to the total variance in the data.
r_squared = r2_score(test_y, predicted_dtree)
r_squared

# This means that the linear regression model explains 67.24% of the variance in the data.

0.9790373552854099

In [13]:
from sklearn.metrics import mean_absolute_error

#If the MAE is 0, this means that the predicted values are perfectly accurate.
# Calculate the accuracy score
mae = mean_absolute_error(test_y, predicted_dtree)
mae

2.8677648964798257

In [17]:
df = pd.DataFrame()
df['likes_dislikes_ratio']= test_y
df['Predicted']=predicted_dtree
df['Error']=df['likes_dislikes_ratio']-df['Predicted']
df

,likes_dislikes_ratio,Predicted,Error
36171,15.440281,15.608035,-0.167754
34553,44.593028,43.442532,1.150495
21934,24.774297,24.711366,0.062931
14719,19.935759,19.237536,0.698223
20114,193.395564,194.085684,-0.690121
...,...,...,...
2853,32.296875,31.790551,0.506324
5261,22.917402,22.515803,0.401599
10130,30.067073,30.655100,-0.588027
8195,239.596914,248.200980,-8.604066


In [18]:
df.sort_values(by='Error', ascending=False)

,likes_dislikes_ratio,Predicted,Error
28120,1252.727778,908.138391,344.589387
2518,1770.838710,1463.236498,307.602211
13015,787.225806,565.400645,221.825161
10995,984.856305,773.011696,211.844609
31955,681.018382,472.085299,208.933084
...,...,...,...
8706,628.070093,777.982848,-149.912755
27154,555.270348,730.035870,-174.765522
34151,284.790359,461.230345,-176.439987
7728,427.962963,612.399329,-184.436366


In [19]:
test_X.loc[[28120]]

,view_count,likes_count,dislikes_count,comment_count,topic1,topic2,topic3
28120,726929,225491,180,17187,3,4,5


In [21]:
pd.options.display.float_format = '{:.2f}'.format
test_X.describe()[['likes_count', 'dislikes_count']]

,likes_count,dislikes_count
count,9316.00,9316.00
mean,166349.75,5229.71
std,489320.25,35394.43
min,97.00,4.00
25%,13361.50,284.00
50%,42380.50,813.00
75%,131037.75,2405.75
max,13641573.00,2397733.00
